Transformation steps in this dataset is to help us align the three datasets in future. 

## Importing libraries

In [2]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from IPython.display import display
from bs4 import BeautifulSoup
import pandas as pd

## Reading from HTML file

In [3]:
maintenance_df=pd.read_html('Datasets/maintenance_df.html', skiprows=1)[0]

In [4]:
maintenance_df.head()

,0,20 Nov 2015,no repair,0.02128,?,0.108,NaN,NaN.1,NaN.2
0,1,"July 24th, 2015",No repair,0.02248,208,0.109,NaN,NaN,NaN
1,2,2017-03-24,repair,0.03639,51,0.119,0.00033,1209.0,0.119
2,3,"July 15th, 2016",No repair,0.01265,566,0.104,NaN,NaN,NaN
3,4,"January 27th, 2017",No repair,0.01429,514,0.116,NaN,NaN,NaN
4,5,20 May 2016,no repair,0.00097,?,0.106,NaN,NaN,NaN


In [5]:
maintenance_df.drop(columns=maintenance_df.columns[0], axis=1, inplace=True)

In [6]:
maintenance_df.columns=['Maintenance date',	'Repair?',	'M1 (before)',	'M2 (before)',	'M3 (before)',	'M1 (after)', 'M2 (after)',	'M3 (after)']

In [7]:
maintenance_df

,Maintenance date,Repair?,M1 (before),M2 (before),M3 (before),M1 (after),M2 (after),M3 (after)
0,"July 24th, 2015",No repair,0.02248,208,0.109,NaN,NaN,NaN
1,2017-03-24,repair,0.03639,51,0.119,0.00033,1209.0,0.119
2,"July 15th, 2016",No repair,0.01265,566,0.104,NaN,NaN,NaN
3,"January 27th, 2017",No repair,0.01429,514,0.116,NaN,NaN,NaN
4,20 May 2016,no repair,0.00097,?,0.106,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
125,2016-04-22,no repair,0.02032,279,0.108,NaN,NaN,NaN
126,2017-03-24,repair,0.03639,51,0.119,0.00033,1209.0,0.119
127,17 Jun 2016,no repair,0.00817,?,0.105,NaN,NaN,NaN
128,16 Oct 2015,repair,0.03707,?,0.113,0.00031,1217.0,0.113


Observations:
- **Maintenance date** is in three different formats. After inquiry, we are informed that there were three different technicians that wrote this log and each put the date in a different style;
- **Repair?** indicates whether there was a repair and recalibration at that maintenance date or not. This one has different spellings and typos as well `"repair"`, `"Repair"` and `"repiar"` all mean there was a repair and recalibration at that maintenance date and `"no repair"` and `"No repair"` both indicate a lack of repair and recalibration;
- If there was a repair done, then the columns **M1 (after)**, **M2 (after)** and **M3 (after)** have values, which are measurements $M_1$, $M_2$ and $M_3$ taken after recalibration, otherwise they are `NaN` ("Not-a-Number");
- Some values of $M_2$ are missing. Upon close examination, it is visible that happens when there is a certain format to the **Maintenance date** column and that means that one of the technicians forgot to measure $M_2$. Also, after inquiry we find out that the technicians each wrote a separate log and this dataset is their logs merged together and one of the technicians did not record $M_2$ measurements in their maintenance log;
- We are also informed that in the event of a repair, $M_1$ and $M_2$ after recalibration, i.e., **M1 (after)** and **M2 (after)**, should just be noisy readings of a fixed value. The expert also tells us that principally, $M_3$ should not be affected by repair, i.e., **M3 (before)** should be equal (with some noise) to **M3 (after)**, if there was a repair. That seems to hold;
- The expert also tells us that there is a strong correlation between values of $M_1$ and $M_2$;
- The different feature values come in different ranges, so it's a good idea to normalize them;
- There are duplicate rows. The duplications are artefacts of the improper merging of the logs written by different technicians;

## Data Vizualization

Every feature except for **Maintenance date** (at column index `0`) seems to be visualizable.

In [8]:
visualizable_feature_names_maintenance = maintenance_df.columns[1:]
num_visualizable_features_maintenance = len(visualizable_feature_names_maintenance)

In [9]:
fig_hist_maintenance = []
for i, feature_name in enumerate(visualizable_feature_names_maintenance):
    fig_hist_maintenance.append(go.Figure(go.Histogram(x=maintenance_df[feature_name])))
    fig_hist_maintenance[i].update_layout(height=400, width=800, title_text=feature_name)
    fig_hist_maintenance[i].show()
    
fig_scatmat_maintenance = go.Figure(data=go.Splom(
                              dimensions=[dict(label=feature, values=maintenance_df[feature]) \
                                  for feature in visualizable_feature_names_maintenance],
                              marker=dict(showscale=False, line_color='white', line_width=0.5)))

fig_scatmat_maintenance.update_layout(title='Pairwise feature scatter plots', \
                                      width=400 * num_visualizable_features_maintenance, \
                                      height=400 * num_visualizable_features_maintenance)

fig_scatmat_maintenance.show()

# Data Preprocessing

### 1. Putting timestamps in the same format

In [10]:
maintenance_dates = maintenance_df["Maintenance date"].values.astype("str")
splitted_dates = np.char.split(maintenance_dates, " ")
triletterorder = {"Jan": "01", "Feb": "02", "Mar": "03", "Apr": "04", "May": "05", "Jun": "06", "Jul": "07", "Aug": "08", "Sep": "09", "Oct": "10", "Nov": "11", "Dec": "12"}
maintenance_dates_formatted = []
for date in splitted_dates:
    if len(date) == 1:
        maintenance_dates_formatted.append(date[0])
    elif date[0].isnumeric():
        maintenance_dates_formatted.append(date[2] + "-" + triletterorder[date[1]] + "-" + date[0].zfill(2))
    else:
        maintenance_dates_formatted.append(date[2] + "-" + triletterorder[date[0][:3]] + "-" + date[1][:-3].zfill(2))
maintenance_df_2 = maintenance_df.copy()
maintenance_df_2["Maintenance date"] = maintenance_dates_formatted

In [11]:
maintenance_df_2

,Maintenance date,Repair?,M1 (before),M2 (before),M3 (before),M1 (after),M2 (after),M3 (after)
0,2015-07-24,No repair,0.02248,208,0.109,NaN,NaN,NaN
1,2017-03-24,repair,0.03639,51,0.119,0.00033,1209.0,0.119
2,2016-07-15,No repair,0.01265,566,0.104,NaN,NaN,NaN
3,2017-01-27,No repair,0.01429,514,0.116,NaN,NaN,NaN
4,2016-05-20,no repair,0.00097,?,0.106,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
125,2016-04-22,no repair,0.02032,279,0.108,NaN,NaN,NaN
126,2017-03-24,repair,0.03639,51,0.119,0.00033,1209.0,0.119
127,2016-06-17,no repair,0.00817,?,0.105,NaN,NaN,NaN
128,2015-10-16,repair,0.03707,?,0.113,0.00031,1217.0,0.113


### 2. Sorting entries by timestamp

In [12]:
maintenance_df_2= maintenance_df_2.sort_values("Maintenance date")

### 3. Fixing misspellings and alternative formattings

In [13]:
repair_column_Repair = (maintenance_df_2["Repair?"] == "Repair")
repair_column_repiar = (maintenance_df_2["Repair?"] == "repiar")
repair_column_No_repair = (maintenance_df_2["Repair?"] == "No repair")

Now, set the value of the portion of `maintenance_df_4` at the intersection of `repair_column_Repair` Boolean series filter and `"Repair?"` column to `"repair"`, the portion of that DataFrame at the intersection of `repair_column_repiar` and `"Repair?"` to `"repair"` as well and set the value of the portion of `maintenance_df_4` at the intersection of `repair_column_No_repair` and `"Repair?"` to `"no repair"`:

In [14]:
maintenance_df_2.loc[repair_column_Repair, "Repair?"] = "reapir"
maintenance_df_2.loc[repair_column_repiar, "Repair?"] = "repair"
maintenance_df_2.loc[repair_column_No_repair, "Repair?"] = "no repair"

This way we have two values for `"Repair?"` column: `"no repair"` and `"repair"`

### 4. Handling missing values

Next up, let's fix the missing values on `"M_2 (before)"` and `"M_2 (after)"`. Although these values happen depending on the value of another column, namely the format of the `"Maintenance date"` from the first DataFrame (which shows which technician did the repair), these values do not have a causal relationship as the technician who examines the device, does not affect the readings of the device. Then, we have missing values which are MCAR. We can use mean (or median or mode or ...) to impute values for $M_2$, however, this might not be the best way. Remember we were informed that there is a strong corrletaion between values of $M_1$ and $M_2$. If we can find a relationship between these values, then maybe we can impute the missing values of $M_2$ from $M_1$ using that relationship. For that, let's plot the two measurements against each other in a plot (from `"M_1 (before)"` and `"M_2 (before)"` columns):

In [15]:
is_m2_before_missing = (maintenance_df_2["M2 (before)"] == "?")
is_m2_before_available = np.logical_not(is_m2_before_missing)
relationship_data = maintenance_df_2[is_m2_before_available][["M1 (before)", "M2 (before)"]].sort_values("M1 (before)").values.astype("float")
x_relationship, y_relationship = relationship_data[:, 0], relationship_data[:, 1]
fig_relationship = go.Figure(data=go.Scatter(x=x_relationship, y=y_relationship))
fig_relationship.update_layout(title="Relationship between M1 and M2", xaxis_title="M1", yaxis_title="M2")
fig_relationship.show()

It looks like $M_1$ might have an logarithmic relationship with $M_2$. Let's try to fit a linear regression between $M_1$ and $\log M_2$ and plot:

In [16]:
from sklearn.linear_model import LinearRegression
y_relationship_log = np.log(y_relationship + 1)

m1_m2_regressor = LinearRegression()
m1_m2_regressor.fit(x_relationship[:, None], y_relationship_log)

y_relationship_predicted = np.exp(m1_m2_regressor.predict(x_relationship[:, None])) - 1
fig_relationship_2 = go.Figure(data=go.Scatter(x=x_relationship, y=y_relationship, name="Actual data"))
fig_relationship_2.add_trace(go.Scatter(x=x_relationship, y=y_relationship_predicted, name="Predicted"))
fig_relationship_2.update_layout(title="Relationship between M1 and M2", xaxis_title="M1", yaxis_title="M2")
fig_relationship_2.show()

Look like we had the right guess. Now, we can use this relationship to impute values for missing values of $M_2$ by predicting using this linear regressor and using appropriate transformations:

In [17]:
m2_missing_m1 = maintenance_df_2[is_m2_before_missing]["M1 (before)"].values.astype("float")
m2_missing_values = (np.exp(m1_m2_regressor.predict(m2_missing_m1[:, None])) - 1).astype("int")
maintenance_df_3 = maintenance_df_2.copy()
maintenance_df_3.loc[is_m2_before_missing, "M2 (before)"] = m2_missing_values

If `"M2 (after)"` had missing values, we could have also used this relationship to predict the missing values for `"M2 (after)"`, using values of `"M1 (after)"`. However, we know that these "after" measurements are just noisy readings of a fixed value, so we might as well just used the average of when that value is available. This is because we had no missing values on `"M2 (after)"`. Howeevr, apparently, that technician (who did not record the $M_2$ measurements) did not happen to do any of the repair and recalibrations.

In [18]:
maintenance_df_4= maintenance_df_3.drop_duplicates()

In [19]:
print("Before:", maintenance_df_3.shape[0], "rows. After: ", maintenance_df_4.shape[0], "rows.")

Before: 130 rows. After:  110 rows.


### 5. Standardization

In [20]:
from sklearn.preprocessing import StandardScaler
maintenance_numerical = maintenance_df_4[["M1 (before)", "M2 (before)", "M3 (before)"]].values.astype("float")

maintenance_numerical_mean = maintenance_numerical.mean(axis=0)
maintenance_numerical_std = maintenance_numerical.std(axis=0)
is_repair = (maintenance_df_4["Repair?"] == "repair")

maintenance_df_7 =  maintenance_df_4.copy()
maintenance_df_7.loc[is_repair, "M1 (after)"] = (maintenance_df_4.loc[is_repair, "M1 (after)"] - maintenance_numerical_mean[0]) / maintenance_numerical_std[0]
maintenance_df_7.loc[is_repair, "M2 (after)"] = (maintenance_df_4.loc[is_repair, "M2 (after)"] - maintenance_numerical_mean[1]) / maintenance_numerical_std[1]
maintenance_df_7.loc[is_repair, "M3 (after)"] = (maintenance_df_4.loc[is_repair, "M3 (after)"] - maintenance_numerical_mean[2]) / maintenance_numerical_std[2]
maintenance_df_7[["M1 (before)", "M2 (before)", "M3 (before)"]] = StandardScaler().fit_transform(maintenance_numerical)

## Inspection

We know that measurements from the device repeat once every seven days when the device went for maintenance. We can take those measurements and copy them seven times, creating a measurement for every day. However, we mayt be able to do something smarter. We knoe that these measurements are indications of the calibration and internal status of the device and change in a continuous fashion.

In [21]:
repetitions = is_repair.astype("int") + 1
num_maintenance_days = maintenance_df_7.shape[0]
x_m = np.repeat(np.arange(num_maintenance_days), repetitions)
y_m1 = np.repeat(maintenance_df_7["M1 (before)"].values, repetitions)
y_m2 = np.repeat(maintenance_df_7["M2 (before)"].values, repetitions)
y_m3 = np.repeat(maintenance_df_7["M3 (before)"].values, repetitions)
y_r = np.repeat(np.zeros((num_maintenance_days,)), repetitions)
repair_indices = (repetitions.cumsum() - 1)[is_repair].values
y_m1[repair_indices] = maintenance_df_7.loc[is_repair, "M1 (after)"].values
y_m2[repair_indices] = maintenance_df_7.loc[is_repair, "M2 (after)"].values
y_m3[repair_indices] = maintenance_df_7.loc[is_repair, "M3 (after)"].values
y_r[repair_indices] = 1
fig_m1 = go.Figure(data=go.Scatter(x=x_m, y=y_m1, name="M1"))
fig_m1.add_trace(go.Scatter(x=x_m, y=y_r, name="Repair"))
fig_m1.update_layout(title="M1 over maintenance days", xaxis_title="Maintenance day index", yaxis_title="M1")
fig_m1.show()
fig_m2 = go.Figure(data=go.Scatter(x=x_m, y=y_m2, name="M2"))
fig_m2.add_trace(go.Scatter(x=x_m, y=y_r, name="Repair"))
fig_m2.update_layout(title="M2 over maintenance days", xaxis_title="Maintenance day index", yaxis_title="M2")
fig_m2.show()
fig_m3 = go.Figure(data=go.Scatter(x=x_m, y=y_m3, name="M3"))
fig_m3.add_trace(go.Scatter(x=x_m, y=y_r, name="Repair"))
fig_m3.update_layout(title="M3 over maintenance days", xaxis_title="Maintenance day index", yaxis_title="M3")
fig_m3.show()

In [22]:
%store maintenance_df_7

Stored 'maintenance_df_7' (DataFrame)


In [23]:
%store is_repair

Stored 'is_repair' (Series)
